# Using PyTigerGraph with Jupyter Notebook

`pyTigerGraph` is a Python library that initiates a secure connection with a TigerGraph REST++ API server.  Among many other actions, this connection allows arbitrary GSQL command execution, as allowed by your user's RBAC privileges.


First install `pyTigerGraph` using `pip` or `conda`

## Use of ConfigParser to protect credentials

Because this notebook is checked in to a public GitHub repository, I place sensitive credentials in a configuration file.  To use this method, also install the `configparser` package.

Create a file called CREDENTIALS.config with sections like
```ini
# keep in .ini format
[DEFAULT]
user = tigergraph
password = Tigergraph
gsPort = 14240
apiPort = 9000
host = http://localhost
# only overrides needed in each section
[northwind]
host = http://1.2.3.4
secret = s123897asd987123
graph = Northwind
token = 123abcdef123
```

In [ ]:
import pyTigerGraph as tg
from configparser import ConfigParser
import pandas as pd

## Initiate a connection to a TigerGraph REST++ API Server

The code below uses 2 `TigerGraphConnection` calls: the first call establishes a relatively unprivileged connection which allows you to generate a token.  If you already have a token from your TigerGraph administrator, you *do not* need to make this first call, and should comment out the line as indicated.

The second connection call uses the token and returns a connection object which will allow all functionality, including making GSQL calls.

In [ ]:
parser = ConfigParser()
_ = parser.read('CREDENTIALS.config') # contains DEFAULT section plus override sections

host = parser.get('northwind', 'host')
username = parser.get('northwind', 'user')
password = parser.get('northwind', 'password')
gsPort = parser.get('northwind', 'gsPort')
apiPort = parser.get('northwind', 'apiPort')
graphName = parser.get('northwind', 'graph')
mysecret = parser.get('northwind', 'secret')
token = parser.get('northwind', 'token')

# First establish a basic connection using a secret.  Do *not* do this if you already have a token
# conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password)
# token = conn.getToken(mysecret, setToken=True, lifetime=None)[0]

# Next use the new token to establish a full access connection for use with GSQL
conn = tg.TigerGraphConnection(host=host, restppPort=apiPort, gsPort=gsPort, graphname=graphName, password=password, apiToken=token)

In [ ]:
# token

In [ ]:
conn.getVertexTypes()

In [ ]:
conn.getEdgeType('located_in')

In [ ]:
# conn.getVertexTypes()
# conn.runInstalledQuery('floors')
print(conn.gsql('show vertex *', options=[]))

In [ ]:
conn.gsql('''
             USE GLOBAL
             CREATE Graph MyTestGraph()''')

In [ ]:
print(conn.gsql('show graph *', options=[]))

In [ ]:
conn.gsql('''
    USE GLOBAL
    DROP GRAPH MyTestGraph''')